In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
dataset = pd.read_csv('../input/natural-disaster/AllNaturalDisasters.csv')
dataset

In [4]:
#dropping columns that are not essential 
dataset.drop(columns  = ['Entity','Code'], inplace = True)

In [29]:
dataset

In [6]:
years= dataset['Year'].to_list()

In [7]:
#extracting year and month from the date column in the dataframe

train_dataset = dataset['Number of reported natural disasters (reported disasters)']

In [8]:
train_dataset = train_dataset
data_lstm = np.asarray(train_dataset)
data_lstm.shape

In [9]:


from sklearn.preprocessing import MinMaxScaler

# Create training and testing sets
ratio = 0.9
input_length = 5    # Here we take 3 years as input length
output_length = 1


#data_lstm.reset_index(inplace=True,drop=True)
#data_lstm = data_lstm.values

# # Scale data between 0 and 1
scaler = MinMaxScaler()
data_lstm = scaler.fit_transform(np.reshape(data_lstm,(-1,1)))
#data_lstm = np.reshape(data_lstm,(-1,1))
# Split between training and testing sets
split = (int)(np.ceil(ratio*len(data_lstm)))
x_train = [data_lstm[i:i+input_length] for i in range(split-input_length)]
y_train = [data_lstm[i+input_length][0] for i in range(split-input_length)]
x_test = [data_lstm[i+split:i+split+input_length] for i in range(len(data_lstm)-split-input_length)]
y_test = [data_lstm[i+split+input_length][0] for i in range(len(data_lstm)-split-input_length)]

# Check shapes and look at some of the values
print(np.shape(x_train),np.shape(y_train))
print(np.shape(x_test),np.shape(y_test))
print(x_train[0])
print(y_train[0])
print(x_test[0])
print(y_test[0])

# Reshape x_train and x_test in order to be used in LSTM layers
x_train_lstm = np.reshape(x_train, (np.shape(x_train)[0], np.shape(x_train)[1], 1))
x_test_lstm = np.reshape(x_test, (np.shape(x_test)[0], np.shape(x_test)[1], 1))

print(np.shape(x_train_lstm))

In [10]:
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [11]:
# Try to predict Average Temperature evolution over next decades regarding land and ocean temperatures

# Import deep learning libraries
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

# Build our model
lstm = Sequential()
 
# Declare the layers
layers = [LSTM(units=8, input_shape=(input_length,1), activation='relu',return_sequences=True),
          LSTM(units=2, activation='relu'),
         Dense(output_length)]
 
# Add the layers to the model
for layer in layers:
    lstm.add(layer)

# Compile our model
lstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
 
# Fit the model
history_lstm = lstm.fit(x_train_lstm, y_train, validation_data=(x_test_lstm,y_test), epochs=20, batch_size=16)

In [12]:
import matplotlib.pyplot as plt
# Plot loss evolution over training
plt.plot(history_lstm.history['loss'])
plt.plot(history_lstm.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')  
plt.xlabel('epochs')
plt.legend(['train','val'], loc='upper left')

In [14]:
# Make some predictions over 50 years and plot them
x_pred = [data_lstm[i:i+input_length] for i in range(len(data_lstm)-input_length)]
x_pred = np.reshape(x_pred,(np.shape(x_pred)[0],np.shape(x_pred)[1],1))

# Make prediction
pred = lstm.predict(x_pred)

In [15]:
pred.shape

In [31]:
# Plot results
plt.figure(figsize=(10,10))
plt.plot(years,data_lstm,color='b')
plt.plot([i for i in range(2015,2050)],pred[-35:],color='r')
plt.legend(['recorded data','prediction'])
plt.xlabel('Years')
plt.ylabel('Average temperature (celsius degree)')
plt.title('Average land and ocean temperature forecast over next 50 years.')
plt.show()

In [45]:
from sklearn import metrics
accuray = metrics.r2_score(data_lstm[55:], pred[50:])

In [46]:
accuray